# Imports

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

import time as t
import os


## Variables

In [ ]:
user = os.getenv("githubUSER")
pswd = os.getenv("githubPSWD")
chromeDRIVER = os.getenv('chromeDRIVER')

base_url = "https://github.com/"


In [ ]:
user

In [ ]:
# Create a webdriver object
# paste path of chrome driver into executable_path in Service
driver = webdriver.Chrome(service=Service(executable_path=chromeDRIVER))

# Exercises

In [ ]:
# Let's open up a website!

driver.get(f'{base_url}login')

In [ ]:
# Enter username

driver.find_element(By.NAME,'login').send_keys(user)

In [ ]:
# Enter password

driver.find_element(By.NAME,'password').send_keys(pswd)

In [ ]:
# Locate and click the sign in button

driver.find_element(By.NAME,'commit').click()

## 1. Make a function that will download all of the sql files listed in your `database-exercises` repo.



In [ ]:
# navigate to database-exercises repo

driver.get(base_url+user+'/database-exercises')

In [ ]:
# //a[@class="js-navigation-open Link--primary"]
driver.find_elements(By.XPATH, '//a[@class="js-navigation-open Link--primary"]')

In [ ]:
#select first sql file

driver.find_elements(By.XPATH, '//a[@class="js-navigation-open Link--primary"]')[1].click()

In [ ]:
# download button

driver.find_element(By.XPATH, '//button[@aria-label="Download raw content"]')

In [ ]:
# Check if this is the correct button by hovering over it
dl_button = driver.find_element(By.XPATH, '//button[@aria-label="Download raw content"]')
ActionChains(driver).move_to_element(dl_button).perform()

In [ ]:
# download the file
# dl_button.click()

In [ ]:
# make sure side panel is open to get next file
try:
    driver.find_element(By.XPATH, '//button[@data-testid="expand-file-tree-button"]').click()
except Exception:
    print('It is already open')

In [ ]:
# find all other files from side panel
driver.find_elements(By.XPATH, '//li[@class="PRIVATE_TreeView-item"]')

In [ ]:
# check len
len(driver.find_elements(By.XPATH, '//li[@class="PRIVATE_TreeView-item"]'))

In [ ]:
# test click next file and hover on download
driver.find_elements(By.XPATH, '//li[@class="PRIVATE_TreeView-item"]')[2].click()
dl_button = driver.find_element(By.XPATH, '//button[@aria-label="Download raw content"]')
ActionChains(driver).move_to_element(dl_button).perform()

### test function parts

In [ ]:
def download_wait(directory:str, timeout:int, n_files:int=None):
    """
    Wait for downloads to finish with a specified timeout.
    
    directory : The path to the folder where the files will be downloaded.
    timeout : How many seconds to wait until timing out.
    n_files : If provided, also wait for the expected number of files.
    """
    # set defaults
    seconds = 0
    dl_wait = True
    total = len(os.listdir(directory)) + n_files
    # loop to check file existences
    while dl_wait and seconds < timeout:
        # sleep
        t.sleep(1)
        # temp false
        dl_wait = False
        # get list of files
        files = os.listdir(directory)
        if len(files) != total:
            dl_wait = True
        # check for unfinished downloads
        for fname in files:
            if fname.endswith('.crdownload'):
                dl_wait = True
        # add to timer
        seconds += 1
    # return seconds

In [ ]:
downloads_dir = os.path.expanduser("~/Downloads")
downloads_dir

In [ ]:
# maximize view to keep side panel open
driver.maximize_window()
# navigate to database-exercises repo
driver.get(base_url+user+'/database-exercises')

# check if first sql file is available
ActionChains(driver).context_click(WebDriverWait(driver,50).until(EC.presence_of_element_located((By.XPATH,'//a[@class="js-navigation-open Link--primary"]')))).perform()

# check for sql files
els = driver.find_elements(By.XPATH, '//a[@class="js-navigation-open Link--primary"]')
sqls = [el for el in els if el.text.endswith('.sql')]
sqls

In [ ]:
#select first sql file
sqls[0].click()


In [ ]:
# download the file
# dl_button = driver.find_element(By.XPATH, '//button[@aria-label="Download raw content"]')
# dl_button.click()
ActionChains(driver).click(WebDriverWait(driver,50).until(EC.presence_of_element_located((By.XPATH,'//button[@aria-label="Download raw content"]')))).perform()
# add timer to wait for download
download_wait(downloads_dir,10,1)

In [ ]:
# make sure side panel is open to get next files
try:
    driver.find_element(By.XPATH, '//button[@data-testid="expand-file-tree-button"]').click()
    # find all other files from side panel
    side_panel = driver.find_elements(By.XPATH, '//li[@class="PRIVATE_TreeView-item"]')
    sqls_panel = [el for el in side_panel if el.text.endswith('.sql')]
except Exception:
    # find all other files from side panel
    side_panel = driver.find_elements(By.XPATH, '//li[@class="PRIVATE_TreeView-item"]')
    sqls_panel = [el for el in side_panel if el.text.endswith('.sql')]
for i in range(1,len(sqls_panel)):
    # check if next sql file is available
    ActionChains(driver).context_click(WebDriverWait(driver,50).until(EC.presence_of_element_located((By.XPATH,'//li[@class="PRIVATE_TreeView-item"]')))).perform()
    # click next file and download
    sqls_panel[i].click()
    # download the file
    # dl_button.click()
    ActionChains(driver).click(WebDriverWait(driver,50).until(EC.presence_of_element_located((By.XPATH,'//button[@aria-label="Download raw content"]')))).perform()
    # add timer to wait for download
    download_wait(downloads_dir,10,1)


In [ ]:
def open_chrome(chromeDRIVER):
    if chromeDRIVER is None:
        return webdriver.Chrome(service=Service())
    return webdriver.Chrome(service=Service(executable_path=chromeDRIVER))


In [ ]:
def download_wait(directory:str, timeout:int, n_files=None):
    """
    Wait for downloads to finish with a specified timeout.
    
    directory : The path to the folder where the files will be downloaded.
    timeout : How many seconds to wait until timing out.
    n_files : If provided, also wait for the expected number of files.
    """
    # set defaults
    seconds = 0
    dl_wait = True
    total = len(os.listdir(directory)) + n_files
    # loop to check file existences
    while dl_wait and seconds < timeout:
        # sleep
        t.sleep(1)
        # temp false
        dl_wait = False
        # get list of files
        files = os.listdir(directory)
        if len(files) != total:
            dl_wait = True
        # check for unfinished downloads
        for fname in files:
            if fname.endswith('.crdownload'):
                dl_wait = True
        # add to timer
        seconds += 1
    # return seconds

In [ ]:
def get_sql_files():
    # open chrome
    driver = open_chrome(chromeDRIVER)
    # sign in to github
    driver.get(f'{base_url}login')
    # Enter username
    driver.find_element(By.NAME,'login').send_keys(user)
    # Enter password
    driver.find_element(By.NAME,'password').send_keys(pswd)
    # Locate and click the sign in button
    driver.find_element(By.NAME,'commit').click()
    # maximize view to keep side panel open
    driver.maximize_window()
    # navigate to database-exercises repo
    driver.get(base_url+user+'/database-exercises')
    # check if first sql file is available
    ActionChains(driver).context_click(WebDriverWait(driver,50).until(EC.presence_of_element_located((By.XPATH,'//a[@class="js-navigation-open Link--primary"]')))).perform()
    # get sql files list
    els = driver.find_elements(By.XPATH, '//a[@class="js-navigation-open Link--primary"]')
    sqls = [el for el in els if el.text.endswith('.sql')]
    next_and_download(sqls, 0)
    # make sure side panel is open to get next files
    try:
        driver.find_element(By.XPATH, '//button[@data-testid="expand-file-tree-button"]').click()
        # find all other files from side panel
        side_panel = driver.find_elements(By.XPATH, '//li[@class="PRIVATE_TreeView-item"]')
        sqls_panel = [el for el in side_panel if el.text.endswith('.sql')]
    except Exception:
        # find all other files from side panel
        side_panel = driver.find_elements(By.XPATH, '//li[@class="PRIVATE_TreeView-item"]')
        sqls_panel = [el for el in side_panel if el.text.endswith('.sql')]
    for i in range(1,len(sqls_panel)):
        # check if next sql file is available
        ActionChains(driver).context_click(WebDriverWait(driver,50).until(EC.presence_of_element_located((By.XPATH,'//li[@class="PRIVATE_TreeView-item"]')))).perform()
        next_and_download(sqls_panel, i)
    driver.close()

def next_and_download(ele_list, i):
    # click next file and download
    ele_list[i].click()
    # download the file
    # dl_button.click()
    ActionChains(driver).click(WebDriverWait(driver,50).until(EC.presence_of_element_located((By.XPATH,'//button[@aria-label="Download raw content"]')))).perform()
    # add timer to wait for download
    download_wait(downloads_dir,10,1)


In [ ]:
driver.quit()

In [ ]:
# test
get_sql_files()

## 2. Pick one more repo to download all ipynb's (**only**) from and turn that into a second function. 



In [ ]:
# maximize view to keep side panel open
driver.maximize_window()


In [ ]:
# navigate to database-exercises repo
driver.get(base_url+user+'/numpy-pandas-visualization-exercises')


In [ ]:
# check if ipynb first file is available

ActionChains(driver).context_click(WebDriverWait(driver,50).until(EC.presence_of_element_located((By.XPATH,'//a[@class="js-navigation-open Link--primary"]')))).perform()


In [ ]:
#select first sql file
driver.find_elements(By.XPATH, '//a[@class="js-navigation-open Link--primary"]')[1].click()


In [ ]:
# download the file
# dl_button = driver.find_element(By.XPATH, '//button[@aria-label="Download raw content"]')
# dl_button.click()
ActionChains(driver).click(WebDriverWait(driver,50).until(EC.presence_of_element_located((By.XPATH,'//button[@aria-label="Download raw content"]')))).perform()


In [ ]:
# make sure side panel is open to get next files
try:
    driver.find_element(By.XPATH, '//button[@data-testid="expand-file-tree-button"]').click()
except Exception:
    # find all other files from side panel
    side_panel = driver.find_elements(By.XPATH, '//li[@class="PRIVATE_TreeView-item"]')
    for i in range(2,len(side_panel)):
        # check if next sql file is available
        ActionChains(driver).context_click(WebDriverWait(driver,50).until(EC.presence_of_element_located((By.XPATH,'//li[@class="PRIVATE_TreeView-item"]')))).perform()
        # click next file and download
        side_panel[i].click()
        # download the file
        # dl_button.click()
        ActionChains(driver).click(WebDriverWait(driver,50).until(EC.presence_of_element_located((By.XPATH,'//button[@aria-label="Download raw content"]')))).perform()

**BONUS**: turn this functions into a py script you can run directly in terminal.